In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 47.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=92a061e941487d3b0f16c0074a0ed2fffe53f025f66ef182853563aed5098966
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName('Questions').getOrCreate()

In [ ]:
spark


# Q.1. Read the given CSV file in a Hive table 



# Q2. Read the data from Hive table as spark dataframe

In [ ]:
df= spark.read.csv('/content/boston.csv',header='true', inferSchema='true')
df.show()

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|18.0|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|       24.0|
|0.027310001| 0.0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729| 0.0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001| 0.0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999| 0.0|2.

# Q3. Get the correlation between dependent and independent variables

In [ ]:
import six
for i in df.columns:
    if not( isinstance(df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to MV for ", i, df.stat.corr('MV',i))

Correlation to MV for  CRIM -0.3883046116575088
Correlation to MV for  ZN 0.36044534463752903
Correlation to MV for  INDUS -0.48372517128143383
Correlation to MV for  CHAS 0.17526017775291847
Correlation to MV for  NOX -0.4273207763683772
Correlation to MV for  RM 0.695359937127267
Correlation to MV for  AGE -0.37695456714288667
Correlation to MV for  DIS 0.24992873873512172
Correlation to MV for  RAD -0.3816262315669168
Correlation to MV for  TAX -0.46853593528654536
Correlation to MV for  PT -0.5077867038116085
Correlation to MV for  B 0.3334608226834164
Correlation to MV for  LSTAT -0.7376627294671615
Correlation to MV for  MV 1.0


In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')


In [ ]:
vhousedf = vectorAssembler.transform(df)


In [ ]:
vhousedf =vhousedf.select(['features', 'MV'])
vhousedf.show()

+--------------------+-----------+
|            features|         MV|
+--------------------+-----------+
|[0.00632,18.0,2.3...|       24.0|
|[0.027310001,0.0,...|21.60000038|
|[0.02729,0.0,7.07...|34.70000076|
|[0.032370001,0.0,...|33.40000153|
|[0.069049999,0.0,...|36.20000076|
|[0.029850001,0.0,...|28.70000076|
|[0.088289998,12.5...|22.89999962|
|[0.144549996,12.5...|27.10000038|
|[0.211239994,12.5...|       16.5|
|[0.170039997,12.5...|18.89999962|
|[0.224889994,12.5...|       15.0|
|[0.117470004,12.5...|18.89999962|
|[0.093780003,12.5...|21.70000076|
|[0.629760027,0.0,...|20.39999962|
|[0.637960017,0.0,...|18.20000076|
|[0.627390027,0.0,...|19.89999962|
|[1.053930044,0.0,...|23.10000038|
|[0.784200013,0.0,...|       17.5|
|[0.802709997,0.0,...|20.20000076|
|[0.725799978,0.0,...|18.20000076|
+--------------------+-----------+
only showing top 20 rows



# Q4. Build a linear regression model to predict house price

In [ ]:
splits = vhousedf.randomSplit([0.7, 0.3])
train= splits[0]
test = splits[1]

In [ ]:
from pyspark.ml.regression import LinearRegression
l = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
model = l.fit(train)
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

Coefficients: [-0.053723746299794956,0.0,0.0,2.0602807489690473,-7.630972560150657,3.762376981390813,0.0,-0.6147167501359809,0.0,0.0,-0.8101066114516472,0.005343288597741152,-0.5566820334740052]
Intercept: 25.570414429043705


#Q5. Evaluate the Linear Regression model by getting the RMSE and R-squared values

In [ ]:
trainsummary = model.summary
print("RMSE: %f" % trainsummary.rootMeanSquaredError)
print("r2: %f" % trainsummary.r2)

RMSE: 5.042771
r2: 0.690415


In [ ]:
train.describe().show()

+-------+-----------------+
|summary|               MV|
+-------+-----------------+
|  count|              341|
|   mean|22.52903231704985|
| stddev|9.076472953206931|
|    min|              5.0|
|    max|             50.0|
+-------+-----------------+



In [ ]:
pred = model.transform(test)
pred.select("prediction","MV","features").show()
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(predictionCol="prediction", 
                 labelCol="MV",metricName="r2")
print("R2 value of test data = %g" % eval.evaluate(pred))

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
|30.641891446541912|       24.0|[0.00632,18.0,2.3...|
| 27.52105514687886|       22.0|[0.01096,55.0,2.2...|
|30.469952973063748|29.10000038|[0.01439,60.0,2.9...|
|27.066878220395914|       24.5|[0.01501,80.0,2.0...|
| 40.09838486928804|       50.0|[0.01501,90.0,1.2...|
| 34.16518637371465|       44.0|[0.01538,90.0,3.7...|
| 26.00449470061726|       33.0|[0.019509999,17.5...|
| 20.36718717414511|20.10000038|[0.019649999,80.0...|
| 38.04991523308212|       50.0|[0.020090001,95.0...|
|26.757480029973486|       16.5|[0.024979999,0.0,...|
|20.233274835052804|       18.5|[0.030409999,0.0,...|
|22.491961783436473|20.60000038|[0.033059999,0.0,...|
| 31.59607438035372|34.90000153|[0.03359,75.0,2.9...|
|24.721680667337782|19.39999962|[0.03466,35.0,6.0...|
| 22.69352515469241|20.89999962|[0.03548,80.0,3.6...|
| 25.69607592123404|24.79999

In [ ]:
re = model.evaluate(test)
print("Root Mean Squared Error (RMSE) of test data = %g" % re.rootMeanSquaredError)

Root Mean Squared Error (RMSE) of test data = 4.67511


In [ ]:
print("number of Iterations: %d" % trainsummary.totalIterations)
print("objective History: %s" % str(trainsummary.objectiveHistory))
trainsummary.residuals.show()

number of Iterations: 10
objective History: [0.5, 0.43461147355104723, 0.24618853800263268, 0.22416027746784492, 0.1969253912938525, 0.19408071331248103, 0.19258482955564177, 0.19192681187627247, 0.19128108115114573, 0.19111806743798157, 0.19080669350042]
+-------------------+
|          residuals|
+-------------------+
| 2.1621368688234206|
| 1.0897448061170998|
|  6.041028364694252|
|  2.017073694931515|
| 12.706440936072212|
| 0.9718435074816298|
|  4.022893467969794|
| 2.4615583396593586|
|-2.5200387252689715|
|-1.0019432413101583|
|  6.647411235572854|
| 0.1657938453761254|
|-3.5108143061414587|
| 3.2949817266537416|
| -4.107663753047955|
| 2.1202483715110816|
|-2.0595009818825503|
| 0.7572241727769686|
| 2.1040388192063624|
| 2.1875704688040045|
+-------------------+
only showing top 20 rows



In [ ]:
pred = model.transform(test)
pred.select("prediction","MV","features").show()

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
|30.641891446541912|       24.0|[0.00632,18.0,2.3...|
| 27.52105514687886|       22.0|[0.01096,55.0,2.2...|
|30.469952973063748|29.10000038|[0.01439,60.0,2.9...|
|27.066878220395914|       24.5|[0.01501,80.0,2.0...|
| 40.09838486928804|       50.0|[0.01501,90.0,1.2...|
| 34.16518637371465|       44.0|[0.01538,90.0,3.7...|
| 26.00449470061726|       33.0|[0.019509999,17.5...|
| 20.36718717414511|20.10000038|[0.019649999,80.0...|
| 38.04991523308212|       50.0|[0.020090001,95.0...|
|26.757480029973486|       16.5|[0.024979999,0.0,...|
|20.233274835052804|       18.5|[0.030409999,0.0,...|
|22.491961783436473|20.60000038|[0.033059999,0.0,...|
| 31.59607438035372|34.90000153|[0.03359,75.0,2.9...|
|24.721680667337782|19.39999962|[0.03466,35.0,6.0...|
| 22.69352515469241|20.89999962|[0.03548,80.0,3.6...|
| 25.69607592123404|24.79999

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'MV')
dt_model = dt.fit(train)
dt_predictions = dt_model.transform(test)
dt_evaluator = RegressionEvaluator(
    labelCol="MV", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) of test data = %g" % rmse)

Root Mean Squared Error (RMSE) of test data = 4.04924
